In [1]:
#오디오 분석을 위한 툴
import sys
from moviepy.editor import *
#경로 읽기 위한 방법
import os

#이미지 분석용
import glob 
import cv2 
import numpy as np 
import pandas as pd 
from tqdm.notebook import tqdm
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from mtcnn.mtcnn import MTCNN

from tensorflow.keras import layers, models, optimizers,Sequential

from tensorflow.keras.layers import Conv3D, ConvLSTM2D, Dense, Flatten, Dropout, BatchNormalization, MaxPooling3D, Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


Using TensorFlow backend.


In [2]:
#분석 사이즈 정의
SEQ_LENGTH = int(10)
IMG_SIZE = int(256)
IMG_SIZE1 = int(256)
LABELS = ['REAL', 'FAKE']
detector=MTCNN()

In [3]:
#이미지 불로온느 기능
def get_sequence(video_path):
#     print(video_path)
    v_cap = cv2.VideoCapture(video_path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
#     samples = np.linspace(0, v_len-1, SEQ_LENGTH+1).round()
    imgs = []
    q=0
    err=0
#     print(v_len)
    for i in range(v_len):
        if(q<10):
            pass
        else:
            break
        v_cap.read()
        v_cap.read()
        v_cap.read()
        if i>(v_len-70):
            finish=True
            while finish==False:
                if(q<10):
                    rel, frame = v_cap.read()
                    if(frame.shape[0]<frame.shape[1]):
                        faces=frame[0:frame.shape[0]-300, 300:frame.shape[1]-300]
                    else:
                        faces=frame[0:frame.shape[0]-600, 150:frame.shape[1]-150]
                    img = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)
#                     img = np.array(img, dtype='uint8')
                    img = cv2.resize(img, None, fx=256, fy=256, interpolation=cv2.INTER_LINEAR) # resize
#                     img = np.mean(img, axis=-1)
#                     img = np.expand_dims(img, axis=-1)
                    imgs.append(img)
                    q=q+1
                else:
                    break
                
                
        rel, frame = v_cap.read()
        if frame is None:
            while True:
                rel, frame = v_cap.read()
                if frame is None:
                    pass
                else:
                    break
                
            
        
        elif i%(1)==0:
            faces = detector.detect_faces(frame)
            if not faces:
                check=True
                while check==True:
                    err=err+1
                    rel, frame = v_cap.read()
                    faces = detector.detect_faces(frame)
                    if not faces:
                        pass
                    else:
                        x, y, width, height = faces[0]['box']
                        x-=40
                        y-=40
                        width+=80
                        height+=80
                        if x<0:
                            x=0
                        if y<0:
                            y=0
                        faces=frame[y:y+height, x:x+width]
                        check=False
                    if err>20:
                        check=False
                        if(frame.shape[0]<frame.shape[1]):
                            faces=frame[0:frame.shape[0]-300, 300:frame.shape[1]-300]
                        else:
                            faces=frame[0:frame.shape[0]-600, 150:frame.shape[1]-150]
            else:
                x, y, width, height = faces[0]['box']
                x-=40
                y-=40
                width+=80
                height+=80
                if x<0:
                    x=0
                if y<0:
                    y=0
                faces=frame[y:y+height, x:x+width]
            if(q<10):
#                 print(type(faces))
#                 print(faces.shape)
#                 print(str(y)+'z'+str(height)+'z'+str(x)+'z'+str(width))
                img = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)
#                 img = np.array(img, dtype='uint8')
                img = cv2.resize(img, (256,256)) # resize
#                 img = np.mean(img, axis=-1)
#                 img = np.expand_dims(img, axis=-1)
#                 print(img.shape)
                imgs.append(img)
#                 print(imgs.shape)
                q=q+1
            else:
                break
            
    
    # update i
    v_cap.release()
    
    return np.array(imgs)

In [4]:
#오디오 데이터 불러오는 기능
def get_audio(video_path):
    path=str(video_path).replace("\\","/")
#     print(path)
#     print(video_path)
    video = VideoFileClip(video_path)
    audio = video.audio
    audiodata=np.random.randn(1,30000,1)
    for i in range(0,1):
        
        for k in range(0,30000):
            audiodata[i][k][0]=audio.get_frame(k/8000)[0]
    
    video.reader.close()
    video.audio.reader.close_proc()
    return audiodata

# video = VideoFileClip('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/train_sample_videos/cbltdtxglo.mp4') # 2.
# audio = video.audio # 3.
# for i in range(0,500):
#     print(audio.get_frame(i/500))
    

In [5]:
#경로 읽는 방법.
# x_val=[]

# for dirname, _, filenames in os.walk('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/meta/dfdc_train_part_49'):

#     for filename in filenames:

#         x_val.append(os.path.join(dirname, filename))

In [6]:
#mixdataget
def mixdataget(video_path):
    a=get_sequence(video_path)
    b=get_audio(video_path)
#     print(a.shape)
#     print(b.shape)
#     mixdata=np.concatenate([a,b],axis=1)
    return a , b

In [7]:
#r경로 읽어 오기
def getpath(target):
    path=[]
#     print(target)
    for dirname, _, filenames in os.walk('D:/Projects/kaggle/deepfake/deepfake-detection-challenge'):
#     for dirname, _, filenames in os.walk('/kaggle/input/deepfake-detection-challenge/test_videos'):

        for filename in filenames:
#             print(filename[-len(target)-1:])
            if filename[-len(target):]==target :            
                path.append(os.path.abspath(os.path.join(dirname, filename)))
    
    try:
        a=path[0]
    except:
        print(target)
    return path[0]


In [8]:
#json file읽어서 데이터 프레임 만들기.

for i in range(1,40):    
    if i==1:
        meta1=pd.read_json('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/meta/ydata/metadata'+str(i)+'.json')
        vn = meta1.columns.values
        vn=np.sort(vn)
        labels = [meta1[n]['label'] for n in vn]
        labels = [LABELS.index(l) for l in labels]
        split = [meta1[n]['split'] for n in vn]
        original=[meta1[n]['original'] for n in vn]
        my_df = pd.DataFrame({"filename":vn, "labels":labels, "split":split, "original":original})
    else:
        meta1=pd.read_json('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/meta/ydata/metadata'+str(i)+'.json')
        vn = meta1.columns.values
        vn=np.sort(vn)
        labels = [meta1[n]['label'] for n in vn]
        labels = [LABELS.index(l) for l in labels]
        split = [meta1[n]['split'] for n in vn]
        original=[meta1[n]['original'] for n in vn]
        my_df1 = pd.DataFrame({"filename":vn, "labels":labels, "split":split, "original":original})
        my_df=pd.concat([my_df,my_df1])
# meta1=pd.read_json('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/metadata1.json')
# vn = meta1.columns.values
# vn=np.sort(vn)
# labels = [meta1[n]['label'] for n in vn]
# labels = [LABELS.index(l) for l in labels]
# split = [meta1[n]['split'] for n in vn]
# original=[meta1[n]['original'] for n in vn]
# my_df = pd.DataFrame({"filename":vn, "labels":labels, "split":split, "original":original})

In [9]:
#랜덤으로 학습 데이터 100 만들어 내기.

def gettraindata():
    ytraindata=np.random.randn(1, 1)
    ny_train=np.random.randn(1, 1)
    traindata=np.random.randn(1,10, 256, 256, 3)
    traindatav=np.random.randn(1,30000,1)
    new=np.random.randn(1,10, 256, 256,3)
    newv=np.random.randn(1,30000,1)
    for i in range(0,300):
        while True:
            data=my_df.sample()
#             print(dat.values)
            if str(data.values[0][1]) == str(1):
                break
        if(i==0):
            try:
                traindata[0,:,:,:,:,],traindatav[0,:,:]=mixdataget(getpath(data.values[0][0]))
                ytraindata[0][0]=1
            except:
                while True:
                    data=my_df.sample()
                    if str(data.values[0][1]) == str(1):
                        break
                traindata[0,:,:,:,:,],traindatav[0,:,:]=mixdataget(getpath(data.values[0][0]))
                ytraindata[0][0]=1
            try:
                new[0,:,:,:,:,],newv[0,:,:]=mixdataget(getpath(data.values[0][3]))
                ny_train[0][0]=0
                ytraindata=np.concatenate([ytraindata, ny_train])
            except:
                while True:
                    data=my_df.sample()
                    if str(data.values[0][1]) == str(1):
                        ny_train[0][0]=1
                        ytraindata=np.concatenate([ytraindata, ny_train])
                        break
                new[0,:,:,:,:,],newv[0,:]=mixdataget(getpath(data.values[0][3]))
            traindata=np.concatenate([traindata,new],axis=0)
            traindatav=np.concatenate([traindatav,newv],axis=0)
        else:
#             print(i)
            try:
                traindata[0,:,:,:,:,],traindatav[0,:,:]=mixdataget(getpath(data.values[0][0]))
            except:
                while True:
                    data=my_df.sample()
                    if str(data.values[0][1]) == str(1):
                        break
                new[0,:,:,:,:,],newv[0,:,:]=mixdataget(getpath(data.values[0][0]))
            traindata=np.concatenate([traindata,new],axis=0)
            traindatav=np.concatenate([traindatav,newv],axis=0)
            ny_train[0][0]=1
            ytraindata=np.concatenate([ytraindata, ny_train])
            try:
                traindata[0,:,:,:,:,],traindatav[0,:,:]=mixdataget(getpath(data.values[0][3]))
                ny_train[0][0]=0
                ytraindata=np.concatenate([ytraindata, ny_train])
            except:
                while True:
                    data=my_df.sample()
                    if str(data.values[0][1]) == str(1):
                        ny_train[0][0]=1
                        ytraindata=np.concatenate([ytraindata, ny_train])
                        break
                new[0,:,:,:,:,],newv[0,:]=mixdataget(getpath(data.values[0][3]))
            traindata=np.concatenate([traindata,new],axis=0)
            traindatav=np.concatenate([traindatav,newv],axis=0)
            print(traindata.shape)
            print(traindatav.shape)
            if(i%10==0):
                print(i)
    
    
            
    
    return traindata,traindatav , ytraindata
    

In [87]:
def define_voicemodel(shape=(30000,1)):
    IMG_SIZE2 = (30000,1)
    input_img2 = layers.Input(shape=(30000,1))
#     start = layers.Conv1D(filters=128, kernel_size=(2))(input_img2)
#     output_imgv14=layers.GlobalMaxPool1D()(start)
    maxdata = layers.MaxPool1D(pool_size=(10),strides=1, padding='same')(input_img2)
    maxdata = layers.Conv1D(filters=4, kernel_size=(1))(maxdata)
    maxdata = layers.BatchNormalization()(maxdata)
    maxdata = layers.Activation('relu')(maxdata)
    averdata = layers.AveragePooling1D(pool_size=(10),strides=1, padding='same')(input_img2)
    averdata = layers.Conv1D(filters=4, kernel_size=(1))(averdata)
    averdata = layers.BatchNormalization()(averdata)
    averdata = layers.Activation('relu')(averdata)
    mindata = -layers.MaxPool1D(pool_size=(10),strides=1, padding='same')(-input_img2)
    mindata = layers.Conv1D(filters=4, kernel_size=(1))(mindata)
    mindata = layers.BatchNormalization()(mindata)
    mindata = layers.Activation('relu')(mindata)
    onv_layerv1 = layers.Conv1D(filters=64, kernel_size=(1))(input_img2)
    onv_layerv1 = layers.BatchNormalization()(onv_layerv1)
    onv_layerv1 = layers.Activation('relu')(onv_layerv1)
    LAH = layers.concatenate([maxdata, averdata,mindata,onv_layerv1,input_img2], axis=2)
    conv_layerv1 = layers.Conv1D(filters=128, kernel_size=(50),strides=10)(LAH)
    conv_layerv1 = layers.BatchNormalization()(conv_layerv1)
    conv_layerv1 = layers.Activation('relu')(conv_layerv1)
    output_imgv13=layers.GlobalMaxPool1D()(conv_layerv1)
    output_imgv13=layers.Dense(128)(output_imgv13)
    output_imgv13 = layers.BatchNormalization()(output_imgv13)
    output_imgv13 = layers.Activation('relu')(output_imgv13)
    add5=layers.Conv1D(filters=4,kernel_size=(20),padding='same')(conv_layerv1)
    add5 = layers.BatchNormalization()(add5)
    add5 = layers.Activation('relu')(add5)
    add5=layers.concatenate([conv_layerv1,add5],axis=2)
    add51=layers.Conv1D(filters=4,kernel_size=(10),padding='same')(add5)
    add51 = layers.BatchNormalization()(add51)
    add51 = layers.Activation('relu')(add51)
    add51=layers.concatenate([conv_layerv1,add5,add51],axis=2)
    add52=layers.Conv1D(filters=4,kernel_size=(5),padding='same')(add51)
    add52 = layers.BatchNormalization()(add52)
    add52 = layers.Activation('relu')(add52)
    add52=layers.concatenate([add5, add51,add52,conv_layerv1], axis=2)
    add53=layers.Conv1D(filters=4,kernel_size=(1))(add52)
    add53 = layers.BatchNormalization()(add53)
    add53 = layers.Activation('relu')(add53)
    add53=layers.concatenate([add5, add51,add52,add53,conv_layerv1], axis=2)
    conv_layerv2 = layers.Conv1D(filters=64, kernel_size=(30),strides=2)(add53)
    conv_layerv2 = layers.BatchNormalization()(conv_layerv2)
    conv_layerv2 = layers.Activation('relu')(conv_layerv2)
    output_imgv12=layers.GlobalMaxPool1D()(conv_layerv2)
    output_imgv12=layers.Dense(128)(output_imgv12)
    output_imgv12 = layers.BatchNormalization()(output_imgv12)
    output_imgv12 = layers.Activation('relu')(output_imgv12)
    add7=layers.Conv1D(filters=16,kernel_size=(10),padding='same')(conv_layerv2)
    add7 = layers.BatchNormalization()(add7)
    add7 = layers.Activation('relu')(add7)
    add7=layers.concatenate([conv_layerv2,add7],axis=2)
    add71=layers.Conv1D(filters=16,kernel_size=(5),padding='same')(add7)
    add71 = layers.BatchNormalization()(add71)
    add71 = layers.Activation('relu')(add71)
    add71=layers.concatenate([conv_layerv2,add7,add71],axis=2)
    add72=layers.Conv1D(filters=16,kernel_size=(2),padding='same')(add71)
    add72 = layers.BatchNormalization()(add72)
    add72 = layers.Activation('relu')(add72)
    add73=layers.concatenate([add7,add71,add72,conv_layerv2],axis=2)
    add74=layers.Conv1D(filters=16,kernel_size=(1))(add73)
    add74 = layers.BatchNormalization()(add74)
    add74 = layers.Activation('relu')(add74)
    add74=layers.concatenate([conv_layerv2,add7,add71,add72,add74],axis=2)
    ## add max pooling to obtain the most imformatic features
    conv_layerv3 = layers.Conv1D(filters=128, kernel_size=( 20),strides=2)(conv_layerv2)
    conv_layerv3 = layers.BatchNormalization()(conv_layerv3)
    conv_layerv3 = layers.Activation('relu')(conv_layerv3)
    output_imgv11=layers.GlobalMaxPool1D()(conv_layerv3)
    output_imgv11=layers.Dense(128)(output_imgv11)
    output_imgv11 = layers.BatchNormalization()(output_imgv11)
    output_imgv11 = layers.Activation('relu')(output_imgv11)
    add6=layers.Conv1D(filters=32,kernel_size=(10),padding='same')(conv_layerv3)
    add6 = layers.BatchNormalization()(add6)
    add6 = layers.Activation('relu')(add6)
    add6=layers.concatenate([conv_layerv3,add6],axis=2)
    add61=layers.Conv1D(filters=32,kernel_size=(5),padding='same')(add6)
    add61 = layers.BatchNormalization()(add61)
    add61 = layers.Activation('relu')(add61)
    add61=layers.concatenate([conv_layerv3,add6,add61],axis=2)
    add62=layers.Conv1D(filters=32,kernel_size=(2),padding='same')(add61)
    add62 = layers.BatchNormalization()(add62)
    add62 = layers.Activation('relu')(add62)
    add63=layers.concatenate([add6,add61,add62,conv_layerv3],axis=2)
    add64=layers.Conv1D(filters=32,kernel_size=(1))(add63)
    add64 = layers.BatchNormalization()(add64)
    add64 = layers.Activation('relu')(add64)
    add64=layers.concatenate([conv_layerv3,add6,add61,add62,add64],axis=2)
    conv_layerv4 = layers.Conv1D(filters=256, kernel_size=(10))(add64)
    conv_layerv4 = layers.BatchNormalization()(conv_layerv4)
    conv_layerv4 = layers.Activation('relu')(conv_layerv4)
    output_imgv1=layers.GlobalMaxPool1D()(conv_layerv4)
    output_imgv2=layers.GlobalAveragePooling1D()(conv_layerv4)
    concatev3 = layers.concatenate([output_imgv1,output_imgv2])
    densei = layers.Dense(512)(concatev3)
    densei = layers.Activation('relu')(densei)
    densei= Dropout(0.8)(densei)
    densei = layers.Dense(32)(concatev3)
    densei = layers.LeakyReLU(alpha=0.1)(densei)
#     densei = layers.Dropout(0.5)(densei)
#     y = layers.Dense(1, activation = 'sigmoid')(densei)
    return input_img2 , densei,output_imgv11,output_imgv12,output_imgv13

In [88]:
def InceptionLayer(a, b, c, d):
    def func(x):
        x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)
        
        x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
        x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)
        x2 = layers.concatenate([x1,x2],axis=3)    
        x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x2)
        x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='relu')(x3)
        x3 = layers.concatenate([x1,x2,x3],axis=3)  
        x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x3)
        x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='relu')(x4)
        y = layers.Concatenate(axis = -1)([x1, x2, x3, x4])
            
        return y
    return func

def define_model(shape=(256,256,3)):
    x = layers.Input(shape = (256,256,3))
    x0 = Conv2D(128, (1, 1), padding='same', activation='relu')(x)
    x0out=layers.GlobalMaxPool2D()(x0)
    x1 = InceptionLayer(4, 16, 16, 8)(x)
    x1 = layers.concatenate([x1,x0],axis=3)  
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
    x1out=layers.GlobalMaxPool2D()(x1)
    
    x2 = InceptionLayer(8, 16, 16, 8)(x1)
    x2 = layers.BatchNormalization()(x2)        
    x2 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
    x2out=layers.GlobalMaxPool2D()(x2)
        
    x3 = layers.Conv2D(64, (5, 5), padding='same', activation = 'relu')(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
    x3out=layers.GlobalMaxPool2D()(x3)
    
    x4 = layers.Conv2D(64, (5, 5), padding='same', activation = 'relu')(x3)
    x4 = layers.BatchNormalization()(x4)
    if shape==(256,256,3):
        x4 = layers.MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
    else:
        x4 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x4)
    y = layers.Flatten()(x4)
    y = layers.Dropout(0.5)(y)
    y = layers.Dense(64)(y)
    y = layers.LeakyReLU(alpha=0.1)(y)
#     y = layers.Dropout(0.5)(y)
#     y = layers.Dense(1, activation = 'sigmoid')(y)

    return x,y,x3out,x2out,x0out


In [102]:

x,y,y01,y02,y03=define_model()
x1,y1,y11,y12,y13=define_model()
x2,y2,y21,y22,y23=define_model()
x3,y3,y31,y32,y33=define_model()
x4,y4,y41,y42,y43=define_model()
x5,y5,y51,y52,y53=define_model()
x6,y6,y61,y62,y63=define_model()
x7,y7,y71,y72,y73=define_model()
x8,y8,y81,y82,y83=define_model()
x9,y9,y91,y92,y93=define_model()
xv,yv,yv1,yv2,yv3=define_voicemodel()
y=layers.concatenate([y,y1,y2,y3,y4,y5,y6,y7,y8,y9,yv])
y1 = layers.Dense(256)(y)
y1 = layers.BatchNormalization()(y1)
y1 = layers.LeakyReLU(alpha=0.1)(y1)
y2 = layers.concatenate([y,y1,y11,y21,y31,y41,y51,y61,yv1,y01])
y2 = layers.Dense(64)(y2)
y2 = layers.BatchNormalization()(y2)
y2 = layers.Activation('relu')(y2)
y3 = layers.concatenate([y,y1,y2,y12,y22,y32,y42,y52,y62,yv2,y02])
y3 = layers.Dense(64)(y3)
y3 = layers.BatchNormalization()(y3)
y3 = layers.Activation('relu')(y3)
y3 = layers.concatenate([y,y1,y2,y3,y13,y23,y33,y43,y53,y63,yv3,y03])
y4 = layers.Dense(128)(y3)
y4 = layers.BatchNormalization()(y4)
y4 = layers.Activation('relu')(y4)
outpp = layers.Dense(1, activation = 'sigmoid')(y3)
modeli = models.Model(inputs=[x,x1,x2,x3,x4,x5,x6,x7,x8,x9,xv], outputs=[outpp])
modeli.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_241 (InputLayer)          [(None, 30000, 1)]   0                                            
__________________________________________________________________________________________________
tf_op_layer_Neg_44 (TensorFlowO [(None, 30000, 1)]   0           input_241[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_45 (MaxPooling1D) (None, 30000, 1)     0           tf_op_layer_Neg_44[0][0]         
__________________________________________________________________________________________________
max_pooling1d_44 (MaxPooling1D) (None, 30000, 1)     0           input_241[0][0]                  
___________________________________________________________________________________________

In [13]:
def makeinputdata(x,x1):
    y=x[0:,0]
    y1=x[0:,1]
    y2=x[0:,2]
    y3=x[0:,3]
    y4=x[0:,4]
    y5=x[0:,5]
    y6=x[0:,6]
    y7=x[0:,7]
    y8=x[0:,8]
    y9=x[0:,9]
    return y,y1,y2,y3,y4,y5,y6,y7,y8,y9,x1

In [103]:
adam = optimizers.Adadelta(lr = 0.00001)
# model.compile(loss = 'mse', optimizer = adam)
modeli.compile(loss = 'binary_crossentropy', optimizer = adam)

In [25]:
for i in range(98,1000):
    x_train,x_train1, y_train=gettraindata()    
    np.save('x6allnewtrain'+str(i),x_train)
    np.save('x6vallnewtrain'+str(i),x_train1)
    np.save('y6allnewtrain'+str(i),y_train)
    history1 = modeli.fit(makeinputdata(x_train,x_train1), y_train, batch_size = 4, shuffle=True,epochs = 20)
    modeli.save('./deepfakeallnew20200606.h5')
    

(4, 10, 256, 256, 3)
(4, 30000, 1)
(6, 10, 256, 256, 3)
(6, 30000, 1)
(8, 10, 256, 256, 3)
(8, 30000, 1)
(10, 10, 256, 256, 3)
(10, 30000, 1)
(12, 10, 256, 256, 3)
(12, 30000, 1)
(14, 10, 256, 256, 3)
(14, 30000, 1)
(16, 10, 256, 256, 3)
(16, 30000, 1)
(18, 10, 256, 256, 3)
(18, 30000, 1)
(20, 10, 256, 256, 3)
(20, 30000, 1)
(22, 10, 256, 256, 3)
(22, 30000, 1)
10
(24, 10, 256, 256, 3)
(24, 30000, 1)
(26, 10, 256, 256, 3)
(26, 30000, 1)
(28, 10, 256, 256, 3)
(28, 30000, 1)
(30, 10, 256, 256, 3)
(30, 30000, 1)
(32, 10, 256, 256, 3)
(32, 30000, 1)
(34, 10, 256, 256, 3)
(34, 30000, 1)
(36, 10, 256, 256, 3)
(36, 30000, 1)
(38, 10, 256, 256, 3)
(38, 30000, 1)
(40, 10, 256, 256, 3)
(40, 30000, 1)
(42, 10, 256, 256, 3)
(42, 30000, 1)
20
(44, 10, 256, 256, 3)
(44, 30000, 1)
(46, 10, 256, 256, 3)
(46, 30000, 1)
(48, 10, 256, 256, 3)
(48, 30000, 1)
(50, 10, 256, 256, 3)
(50, 30000, 1)
(52, 10, 256, 256, 3)
(52, 30000, 1)
(54, 10, 256, 256, 3)
(54, 30000, 1)
(56, 10, 256, 256, 3)
(56, 30000, 1)
(

(426, 10, 256, 256, 3)
(426, 30000, 1)
(428, 10, 256, 256, 3)
(428, 30000, 1)
(430, 10, 256, 256, 3)
(430, 30000, 1)
(432, 10, 256, 256, 3)
(432, 30000, 1)
(434, 10, 256, 256, 3)
(434, 30000, 1)
(436, 10, 256, 256, 3)
(436, 30000, 1)
(438, 10, 256, 256, 3)
(438, 30000, 1)
(440, 10, 256, 256, 3)
(440, 30000, 1)
(442, 10, 256, 256, 3)
(442, 30000, 1)
220
(444, 10, 256, 256, 3)
(444, 30000, 1)
(446, 10, 256, 256, 3)
(446, 30000, 1)
(448, 10, 256, 256, 3)
(448, 30000, 1)
(450, 10, 256, 256, 3)
(450, 30000, 1)
(452, 10, 256, 256, 3)
(452, 30000, 1)
(454, 10, 256, 256, 3)
(454, 30000, 1)
(456, 10, 256, 256, 3)
(456, 30000, 1)
(458, 10, 256, 256, 3)
(458, 30000, 1)
(460, 10, 256, 256, 3)
(460, 30000, 1)
(462, 10, 256, 256, 3)
(462, 30000, 1)
230
(464, 10, 256, 256, 3)
(464, 30000, 1)
(466, 10, 256, 256, 3)
(466, 30000, 1)
(468, 10, 256, 256, 3)
(468, 30000, 1)
(470, 10, 256, 256, 3)
(470, 30000, 1)
(472, 10, 256, 256, 3)
(472, 30000, 1)
(474, 10, 256, 256, 3)
(474, 30000, 1)
(476, 10, 256, 25

(162, 10, 256, 256, 3)
(162, 30000, 1)
80
(164, 10, 256, 256, 3)
(164, 30000, 1)
(166, 10, 256, 256, 3)
(166, 30000, 1)
(168, 10, 256, 256, 3)
(168, 30000, 1)
(170, 10, 256, 256, 3)
(170, 30000, 1)
(172, 10, 256, 256, 3)
(172, 30000, 1)
(174, 10, 256, 256, 3)
(174, 30000, 1)
(176, 10, 256, 256, 3)
(176, 30000, 1)
(178, 10, 256, 256, 3)
(178, 30000, 1)
(180, 10, 256, 256, 3)
(180, 30000, 1)
(182, 10, 256, 256, 3)
(182, 30000, 1)
90
(184, 10, 256, 256, 3)
(184, 30000, 1)
(186, 10, 256, 256, 3)
(186, 30000, 1)
(188, 10, 256, 256, 3)
(188, 30000, 1)
(190, 10, 256, 256, 3)
(190, 30000, 1)
(192, 10, 256, 256, 3)
(192, 30000, 1)
(194, 10, 256, 256, 3)
(194, 30000, 1)
(196, 10, 256, 256, 3)
(196, 30000, 1)
(198, 10, 256, 256, 3)
(198, 30000, 1)
(200, 10, 256, 256, 3)
(200, 30000, 1)
(202, 10, 256, 256, 3)
(202, 30000, 1)
100
(204, 10, 256, 256, 3)
(204, 30000, 1)
(206, 10, 256, 256, 3)
(206, 30000, 1)
(208, 10, 256, 256, 3)
(208, 30000, 1)
(210, 10, 256, 256, 3)
(210, 30000, 1)
(212, 10, 256, 

(578, 10, 256, 256, 3)
(578, 30000, 1)
(580, 10, 256, 256, 3)
(580, 30000, 1)
(582, 10, 256, 256, 3)
(582, 30000, 1)
290
(584, 10, 256, 256, 3)
(584, 30000, 1)
(586, 10, 256, 256, 3)
(586, 30000, 1)
(588, 10, 256, 256, 3)
(588, 30000, 1)
(590, 10, 256, 256, 3)
(590, 30000, 1)
(592, 10, 256, 256, 3)
(592, 30000, 1)
(594, 10, 256, 256, 3)
(594, 30000, 1)
(596, 10, 256, 256, 3)
(596, 30000, 1)
(598, 10, 256, 256, 3)
(598, 30000, 1)
(600, 10, 256, 256, 3)
(600, 30000, 1)
Train on 600 samples
Epoch 1/20
600/600 [==============================] - 44s 73ms/sample - loss: 0.7755
Epoch 2/20
600/600 [==============================] - 43s 72ms/sample - loss: 0.7679
Epoch 3/20
600/600 [==============================] - 44s 73ms/sample - loss: 0.7382
Epoch 4/20
600/600 [==============================] - 44s 73ms/sample - loss: 0.7273
Epoch 5/20
600/600 [==============================] - 44s 73ms/sample - loss: 0.7085
Epoch 6/20
600/600 [==============================] - 44s 73ms/sample - loss: 0.71

(314, 10, 256, 256, 3)
(314, 30000, 1)
(316, 10, 256, 256, 3)
(316, 30000, 1)
(318, 10, 256, 256, 3)
(318, 30000, 1)
(320, 10, 256, 256, 3)
(320, 30000, 1)
(322, 10, 256, 256, 3)
(322, 30000, 1)
160
(324, 10, 256, 256, 3)
(324, 30000, 1)
(326, 10, 256, 256, 3)
(326, 30000, 1)
(328, 10, 256, 256, 3)
(328, 30000, 1)
(330, 10, 256, 256, 3)
(330, 30000, 1)
(332, 10, 256, 256, 3)
(332, 30000, 1)
(334, 10, 256, 256, 3)
(334, 30000, 1)
(336, 10, 256, 256, 3)
(336, 30000, 1)
(338, 10, 256, 256, 3)
(338, 30000, 1)
(340, 10, 256, 256, 3)
(340, 30000, 1)
(342, 10, 256, 256, 3)
(342, 30000, 1)
170
(344, 10, 256, 256, 3)
(344, 30000, 1)
(346, 10, 256, 256, 3)
(346, 30000, 1)
(348, 10, 256, 256, 3)
(348, 30000, 1)
(350, 10, 256, 256, 3)
(350, 30000, 1)
(352, 10, 256, 256, 3)
(352, 30000, 1)
(354, 10, 256, 256, 3)
(354, 30000, 1)
(356, 10, 256, 256, 3)
(356, 30000, 1)
(358, 10, 256, 256, 3)
(358, 30000, 1)
(360, 10, 256, 256, 3)
(360, 30000, 1)
(362, 10, 256, 256, 3)
(362, 30000, 1)
180
(364, 10, 256

...

(300, 10, 256, 256, 3)
(300, 30000, 1)
(302, 10, 256, 256, 3)
(302, 30000, 1)
150
(304, 10, 256, 256, 3)
(304, 30000, 1)
(306, 10, 256, 256, 3)
(306, 30000, 1)
(308, 10, 256, 256, 3)
(308, 30000, 1)
(310, 10, 256, 256, 3)
(310, 30000, 1)
(312, 10, 256, 256, 3)
(312, 30000, 1)
(314, 10, 256, 256, 3)
(314, 30000, 1)
(316, 10, 256, 256, 3)
(316, 30000, 1)
(318, 10, 256, 256, 3)
(318, 30000, 1)
(320, 10, 256, 256, 3)
(320, 30000, 1)
(322, 10, 256, 256, 3)
(322, 30000, 1)
160
(324, 10, 256, 256, 3)
(324, 30000, 1)
(326, 10, 256, 256, 3)
(326, 30000, 1)
(328, 10, 256, 256, 3)
(328, 30000, 1)
(330, 10, 256, 256, 3)
(330, 30000, 1)
(332, 10, 256, 256, 3)
(332, 30000, 1)
(334, 10, 256, 256, 3)
(334, 30000, 1)
(336, 10, 256, 256, 3)
(336, 30000, 1)
(338, 10, 256, 256, 3)
(338, 30000, 1)
(340, 10, 256, 256, 3)
(340, 30000, 1)
(342, 10, 256, 256, 3)
(342, 30000, 1)
170
(344, 10, 256, 256, 3)
(344, 30000, 1)
(346, 10, 256, 256, 3)
(346, 30000, 1)
(348, 10, 256, 256, 3)
(348, 30000, 1)
(350, 10, 256

(32, 10, 256, 256, 3)
(32, 30000, 1)
(34, 10, 256, 256, 3)
(34, 30000, 1)
(36, 10, 256, 256, 3)
(36, 30000, 1)
(38, 10, 256, 256, 3)
(38, 30000, 1)
(40, 10, 256, 256, 3)
(40, 30000, 1)
(42, 10, 256, 256, 3)
(42, 30000, 1)
20
(44, 10, 256, 256, 3)
(44, 30000, 1)
(46, 10, 256, 256, 3)
(46, 30000, 1)
(48, 10, 256, 256, 3)
(48, 30000, 1)
(50, 10, 256, 256, 3)
(50, 30000, 1)
(52, 10, 256, 256, 3)
(52, 30000, 1)
(54, 10, 256, 256, 3)
(54, 30000, 1)
(56, 10, 256, 256, 3)
(56, 30000, 1)
(58, 10, 256, 256, 3)
(58, 30000, 1)
(60, 10, 256, 256, 3)
(60, 30000, 1)
(62, 10, 256, 256, 3)
(62, 30000, 1)
30
(64, 10, 256, 256, 3)
(64, 30000, 1)
(66, 10, 256, 256, 3)
(66, 30000, 1)
(68, 10, 256, 256, 3)
(68, 30000, 1)
(70, 10, 256, 256, 3)
(70, 30000, 1)
(72, 10, 256, 256, 3)
(72, 30000, 1)
(74, 10, 256, 256, 3)
(74, 30000, 1)
(76, 10, 256, 256, 3)
(76, 30000, 1)
(78, 10, 256, 256, 3)
(78, 30000, 1)
(80, 10, 256, 256, 3)
(80, 30000, 1)
(82, 10, 256, 256, 3)
(82, 30000, 1)
40
(84, 10, 256, 256, 3)
(84, 30

(452, 10, 256, 256, 3)
(452, 30000, 1)
(454, 10, 256, 256, 3)
(454, 30000, 1)
(456, 10, 256, 256, 3)
(456, 30000, 1)
(458, 10, 256, 256, 3)
(458, 30000, 1)
(460, 10, 256, 256, 3)
(460, 30000, 1)
(462, 10, 256, 256, 3)
(462, 30000, 1)
230
(464, 10, 256, 256, 3)
(464, 30000, 1)
(466, 10, 256, 256, 3)
(466, 30000, 1)
(468, 10, 256, 256, 3)
(468, 30000, 1)
(470, 10, 256, 256, 3)
(470, 30000, 1)
(472, 10, 256, 256, 3)
(472, 30000, 1)
(474, 10, 256, 256, 3)
(474, 30000, 1)
(476, 10, 256, 256, 3)
(476, 30000, 1)
(478, 10, 256, 256, 3)
(478, 30000, 1)
(480, 10, 256, 256, 3)
(480, 30000, 1)
(482, 10, 256, 256, 3)
(482, 30000, 1)
240
(484, 10, 256, 256, 3)
(484, 30000, 1)
(486, 10, 256, 256, 3)
(486, 30000, 1)
(488, 10, 256, 256, 3)
(488, 30000, 1)
(490, 10, 256, 256, 3)
(490, 30000, 1)
(492, 10, 256, 256, 3)
(492, 30000, 1)
(494, 10, 256, 256, 3)
(494, 30000, 1)
(496, 10, 256, 256, 3)
(496, 30000, 1)
(498, 10, 256, 256, 3)
(498, 30000, 1)
(500, 10, 256, 256, 3)
(500, 30000, 1)
(502, 10, 256, 25

(188, 10, 256, 256, 3)
(188, 30000, 1)
(190, 10, 256, 256, 3)
(190, 30000, 1)
(192, 10, 256, 256, 3)
(192, 30000, 1)
(194, 10, 256, 256, 3)
(194, 30000, 1)
(196, 10, 256, 256, 3)
(196, 30000, 1)
(198, 10, 256, 256, 3)
(198, 30000, 1)
(200, 10, 256, 256, 3)
(200, 30000, 1)
(202, 10, 256, 256, 3)
(202, 30000, 1)
100
(204, 10, 256, 256, 3)
(204, 30000, 1)
(206, 10, 256, 256, 3)
(206, 30000, 1)
(208, 10, 256, 256, 3)
(208, 30000, 1)
(210, 10, 256, 256, 3)
(210, 30000, 1)
(212, 10, 256, 256, 3)
(212, 30000, 1)
(214, 10, 256, 256, 3)
(214, 30000, 1)
(216, 10, 256, 256, 3)
(216, 30000, 1)
(218, 10, 256, 256, 3)
(218, 30000, 1)
(220, 10, 256, 256, 3)
(220, 30000, 1)
(222, 10, 256, 256, 3)
(222, 30000, 1)
110
(224, 10, 256, 256, 3)
(224, 30000, 1)
(226, 10, 256, 256, 3)
(226, 30000, 1)
(228, 10, 256, 256, 3)
(228, 30000, 1)
(230, 10, 256, 256, 3)
(230, 30000, 1)
(232, 10, 256, 256, 3)
(232, 30000, 1)
(234, 10, 256, 256, 3)
(234, 30000, 1)
(236, 10, 256, 256, 3)
(236, 30000, 1)
(238, 10, 256, 25

OSError: 1179648000 requested and 0 written

In [104]:
history1 = modeli.fit(makeinputdata(x_train,x_train1), y_train, batch_size = 2, shuffle=True,epochs = 200)

Train on 600 samples
Epoch 1/200
600/600 [==============================] - 101s 169ms/sample - loss: 1.4898
Epoch 2/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.4149
Epoch 3/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.3332
Epoch 4/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.1981
Epoch 5/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0738
Epoch 6/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.1432
Epoch 7/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0744
Epoch 8/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0615
Epoch 9/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0433
Epoch 10/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0919
Epoch 11/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0191
Epoch 12/200
600/60

Epoch 94/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9941
Epoch 95/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.8851
Epoch 96/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9591
Epoch 97/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0241
Epoch 98/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9987
Epoch 99/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9976
Epoch 100/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9261
Epoch 101/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0054
Epoch 102/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0152
Epoch 103/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0031
Epoch 104/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9347
Epoch 105/200
600/600 [====

600/600 [==============================] - 88s 147ms/sample - loss: 0.9904
Epoch 187/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9118
Epoch 188/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0595
Epoch 189/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9718
Epoch 190/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.8991
Epoch 191/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.8946
Epoch 192/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9926
Epoch 193/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9861
Epoch 194/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9659
Epoch 195/200
600/600 [==============================] - 88s 147ms/sample - loss: 0.9667
Epoch 196/200
600/600 [==============================] - 88s 147ms/sample - loss: 1.0653
Epoch 197/200
600/600 [============

In [ ]:
np.save('xallnewtrain44464',x_train)
np.save('yallnewtrain44464',y_train)

In [7]:
b=np.random.randn(1, 300, 300, 1)

In [32]:
b=get_audio('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/train_sample_videos/cbltdtxglo.mp4')

In [101]:
test=modeli.predict(makeinputdata(x_train[30:35],x_train1[30:35]))
test
# test=modeli.predict([x_train[10:15],x_train1[10:15]])

array([[0.37157938],
       [0.37157938],
       [0.37157938],
       [0.37157938],
       [0.37157938]], dtype=float32)

In [35]:
data=my_df.sample()
type(data.values[0][1])

int

In [24]:
xtest.shape
xvtest.shape
ytest.shape

(1800, 1)

In [16]:

ytest=np.random.randn(600, 1)
yadd=np.random.randn(600, 1)
xtest=np.random.randn(600,10, 150, 150, 3)
xvtest=np.random.randn(600,30000,1)
xadd=np.random.randn(600,10, 150, 150,3)
xvadd=np.random.randn(600,30000,1)


xtest=np.load('x5allnewtrain10.npy')
xvtest=np.load('x5vallnewtrain10.npy')
ytest=np.load('y5allnewtrain10.npy')

for i in range(1,3):
    xadd=np.load('x5allnewtrain'+str(i)+'.npy')
    xvadd=np.load('x5vallnewtrain'+str(i)+'.npy')
    yadd=np.load('y5allnewtrain'+str(i)+'.npy')
    xtest=np.concatenate((xtest,xadd),axis=0)
    xvtest=np.concatenate((xvtest,xvadd),axis=0)
    ytest=np.concatenate((ytest,yadd),axis=0)


In [ ]:
adam = optimizers.Adam(lr = 0.000001)
# model.compile(loss = 'mse', optimizer = adam)
modeli.compile(loss = 'binary_crossentropy', optimizer = adam)
history1 = modeli.fit(makeinputdata(xtest,xvtest), ytest, batch_size = 5, shuffle=True,epochs = 2000)

Train on 1800 samples
Epoch 1/2000
1800/1800 [==============================] - 51s 28ms/sample - loss: 0.7341
Epoch 2/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7261
Epoch 3/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7108
Epoch 4/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7285
Epoch 5/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7205s - loss: 0
Epoch 6/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7195
Epoch 7/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7199
Epoch 8/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7142
Epoch 9/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7173
Epoch 10/2000
1800/1800 [==============================] - 38s 21ms/sample - loss: 0.7127
Epoch 11/2000
1800/1800 [==============================] - 39s 21ms/sample -

1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6951
Epoch 180/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.7024
Epoch 181/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.7002
Epoch 182/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.7017
Epoch 183/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6974
Epoch 184/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6959
Epoch 185/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6964
Epoch 186/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.7025
Epoch 187/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6967
Epoch 188/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6967
Epoch 189/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6960
Epoch 190/2000

1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6960
Epoch 355/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6947
Epoch 356/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6966
Epoch 357/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6953
Epoch 358/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6963
Epoch 359/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6972
Epoch 360/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6944
Epoch 361/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6954
Epoch 362/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6955
Epoch 363/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6944s - los 
Epoch 364/2000
1800/1800 [==============================] - 36s 20ms/sample - loss: 0.6944
Epoch 

In [33]:
test=modeli.predict([xtest[10:15],xvtest[10:15]])
test

NameError: name 'xtest' is not defined

In [114]:
adam = optimizers.SGD(lr = 0.0001)
# model.compile(loss = 'mse', optimizer = adam)
modeli.compile(loss = 'binary_crossentropy', optimizer = adam)
history1 = modeli.fit(makeinputdata(x_train,x_train1), y_train, batch_size = 5, shuffle=True,epochs = 200)
# history1 = modeli.fit([x_train,x_train1], y_train, batch_size = 5, shuffle=True,epochs = 20)

MemoryError: Unable to allocate 309. MiB for an array with shape (600, 150, 150, 3) and data type float64

In [40]:
labels1 = [meta1[n]['label'] for n in vn]

In [10]:
test=gettraindata()

wyuwbkywwa.mp4
D:\Projects\kaggle\deepfake\deepfake-detection-challenge\dfdc_train_part_1\wyuwbkywwa.mp4
D:/Projects/kaggle/deepfake/deepfake-detection-challenge/dfdc_train_part_1/wyuwbkywwa.mp4
wyuwbkywwa.mp4
uonwfowmfa.mp4
D:\Projects\kaggle\deepfake\deepfake-detection-challenge\dfdc_train_part_1\uonwfowmfa.mp4
D:/Projects/kaggle/deepfake/deepfake-detection-challenge/dfdc_train_part_1/uonwfowmfa.mp4


In [12]:
test.shape

(2, 20, 315, 300, 1)

In [164]:
video = VideoFileClip('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/dfdc_train_part_1/lpmjgfjpec.mp4')
audio = video.audio

PermissionError: [WinError 5] 액세스가 거부되었습니다

In [165]:
video = VideoFileClip('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/train_sample_videos/cbltdtxglo.mp4') # 2.
audio = video.audio # 3.
for i in range(0,500):
    print(audio.get_frame(i/500))

PermissionError: [WinError 5] 액세스가 거부되었습니다

In [71]:
v_cap = cv2.VideoCapture('D:/Projects/kaggle/deepfake/deepfake-detection-challenge/test_videos/ajiyrjfyzp.mp4')

In [72]:
rel, frame = v_cap.read()

In [85]:
faces = fast_mtcnn(frame)

TypeError: 'NoneType' object is not subscriptable

In [87]:
d=MTCNN()

In [88]:
faces = d(frame)

In [90]:
faces.shape

torch.Size([3, 160, 160])

In [170]:
video.reader.close()
video.audio.reader.close_proc()

NameError: name 'video' is not defined